In [2]:
import polars as pl
import numpy as np

In [3]:
df = pl.read_parquet("FD.parquet")
names = df.columns

In [4]:
def add_timesteps(df:pl.DataFrame)->pl.DataFrame:
    df = df.with_columns(pl.int_range(1,24_001).alias('timestep'))
    return df

In [5]:
def melt_df(df):
    df_melted = df.melt(id_vars=['timestep'], variable_name='case', value_name='value')
    df_melted = df_melted.filter(
    pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
    )
    return df_melted


In [6]:
def add_kt_um(df_melted:pl.DataFrame)->pl.DataFrame:
    dfm = df_melted.with_columns(
    pl.col("case").str.slice(0,4).cast(pl.Float64).alias("kT"),
    pl.col("case").str.slice(5,6).cast(pl.UInt16).alias("um"),
    )
    return dfm
    

In [7]:
df = add_timesteps(df)
df_melted = melt_df(df)
dfm = add_kt_um(df_melted)

C:\Users\zafi_\AppData\Local\Temp\ipykernel_9208\2691555746.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melted = df.melt(id_vars=['timestep'], variable_name='case', value_name='value')


In [8]:
from lets_plot import *
LetsPlot.setup_html()

In [9]:
from scipy.optimize import curve_fit

In [10]:
def exp_decay(x,a,b):
    return a*np.exp(-x*b)

In [11]:
names = df.columns
names.remove("timestep")
names

['2.80_10',
 '2.80_20',
 '2.80_40',
 '2.80_60',
 '3.00_10',
 '3.00_20',
 '3.00_40',
 '3.00_60',
 '3.50_10',
 '3.50_20',
 '3.50_40',
 '3.50_60',
 '4.00_10',
 '4.00_20',
 '4.00_40',
 '4.00_60']

In [12]:
def make_fits(df:pl.DataFrame, max_timestep:int)->pl.DataFrame:
    names = df.columns
    names.remove("timestep")
    time = df.select(pl.col("timestep")).to_numpy()
    fits = pl.DataFrame(pl.Series(time[:max_timestep].flatten()).alias("timestep"))
    rates = pl.DataFrame()
    for name in names:
        val = df.select(pl.col(name)).to_numpy()

        t = time[~np.isnan(val)]
        val = val[~np.isnan(val)]    
        
        popt, _= curve_fit(exp_decay, t, val)

        b = popt[1]
        print(f"Rate for {name} = {b}")
        rates = rates.with_columns(pl.Series([b]).alias(name))

        y_fit = exp_decay(time[:max_timestep].flatten(), *popt)
        y_fit[y_fit<0.1] = np.nan

        fits = fits.with_columns(pl.Series(y_fit).alias(name))

    return fits, rates

In [13]:
fits,rates = make_fits(df,max_timestep=dfm.select("timestep").to_numpy().max())
fits_melted = melt_df(fits)
fitsm = add_kt_um(fits_melted)

Rate for 2.80_10 = 0.11102630324726778
Rate for 2.80_20 = 0.06658834399281063
Rate for 2.80_40 = 0.08080294559825568
Rate for 2.80_60 = 0.07580421163663527
Rate for 3.00_10 = 0.1464036720768182
Rate for 3.00_20 = 0.10243332258722451
Rate for 3.00_40 = 0.08821955584838222
Rate for 3.00_60 = 0.08734601973181542
Rate for 3.50_10 = 0.27605780584517997
Rate for 3.50_20 = 0.150285087168788
Rate for 3.50_40 = 0.11202715212866624
Rate for 3.50_60 = 0.16822597175742224
Rate for 4.00_10 = 0.25218290402658305
Rate for 4.00_20 = 0.2797520471013783
Rate for 4.00_40 = 0.14784089686187768
Rate for 4.00_60 = 0.13277728504752115


C:\Users\zafi_\AppData\Local\Temp\ipykernel_9208\1279754375.py:2: RuntimeWarning: overflow encountered in exp
  return a*np.exp(-x*b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_9208\1279754375.py:2: RuntimeWarning: overflow encountered in multiply
  return a*np.exp(-x*b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_9208\2691555746.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melted = df.melt(id_vars=['timestep'], variable_name='case', value_name='value')


In [14]:
fits

timestep,2.80_10,2.80_20,2.80_40,2.80_60,3.00_10,3.00_20,3.00_40,3.00_60,3.50_10,3.50_20,3.50_40,3.50_60,4.00_10,4.00_20,4.00_40,4.00_60
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,99.601358,93.448976,96.916712,91.083518,96.83437,98.629492,96.052381,94.049327,102.352196,92.871034,88.105692,93.273066,97.51319,95.758004,90.342122,82.924259
2,89.13477,87.429018,89.393594,84.434211,83.646394,89.02676,87.941702,86.183038,77.66168,79.912055,78.768249,78.830929,75.777752,72.39029,77.926254,72.613464
3,79.768061,81.796863,82.454455,78.270318,72.254503,80.358967,80.515891,78.974685,58.927281,68.761337,70.420388,66.624971,58.887087,54.724971,67.21672,63.584713
4,71.385651,76.52753,76.053963,72.556403,62.414085,72.535084,73.717116,72.36924,44.712198,59.166561,62.957234,56.308949,45.761308,41.3705,57.979015,55.678596
5,63.884105,71.597647,70.150307,67.259618,53.913845,65.472947,67.492431,66.316274,33.926233,50.910615,56.285026,47.590231,35.561231,31.274905,50.010862,48.755524
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
fitsm

timestep,case,value,kT,um
i64,str,f64,f64,u16
1,"""2.80_10""",99.601358,2.8,10
2,"""2.80_10""",89.13477,2.8,10
3,"""2.80_10""",79.768061,2.8,10
4,"""2.80_10""",71.385651,2.8,10
5,"""2.80_10""",63.884105,2.8,10
…,…,…,…,…
47,"""4.00_60""",0.184551,4.0,60
48,"""4.00_60""",0.161604,4.0,60
49,"""4.00_60""",0.14151,4.0,60


In [35]:
def make_graph(kt = 3.5):

    selected_vals = dfm.filter(pl.col("kT")==kt)
    selected_fits = fitsm.filter(pl.col("kT")==kt)

    p = (
        ggplot()
        + geom_point(data=selected_vals, mapping=aes(x="timestep", y="value", color=as_discrete("um") ),size=3)
        + geom_line(data=selected_fits, mapping=aes(x="timestep", y="value", color=as_discrete("um") ),size=1.5)
        + scale_color_viridis()
        #+ scale_y_log10()
        + theme_classic()
        + labs(title=f"{kt}kT")
    )
    return p

In [37]:
graphs = list()
KTS = ["2.80", "3.00", "3.50", "4.00"]

grid = gggrid([make_graph(float(x)) for x in KTS], ncol=2)
grid

In [38]:
grid.to_png("decays.png")

'c:\\Users\\zafi_\\paper\\residence2\\6_FD\\decays.png'

: 

In [24]:
ratesm = add_kt_um(rates.melt(variable_name='case', value_name='value'))

C:\Users\zafi_\AppData\Local\Temp\ipykernel_9208\2355123447.py:1: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  ratesm = add_kt_um(rates.melt(variable_name='case', value_name='value'))


In [19]:
rates_plot = (
    ggplot(ratesm,aes(x=as_discrete("kT"),y="value",fill=as_discrete("um")))
    + geom_bar(stat="identity",position="dodge",color="#3f3f3f")
    + scale_fill_brewer("seq")
    + labs(y="k (rate)")
)

In [21]:
rates_plot

In [20]:
rates_plot.to_png("rates.png")

'c:\\Users\\zafi_\\paper\\residence2\\6_FD\\rates.png'